# Simple CNN
- ゼロから作るDeepLearning掲載の簡単なCNN
- Conv-ReLU-Pooling-Dense-ReLU-Dense-Softmaxという構造

In [17]:
import numpy as np
import tensorflow as tf
import keras
import sys, os
sys.path.append(os.pardir)
from collections import OrderedDict
from common.layers import Convolution, Relu, Pooling, Affine, SoftmaxWithLoss
from common.util import im2col
import matplotlib.pyplot as plt

%matplotlib inline

In [15]:
class ScratchSimpleCNN:
    
    # constructer
    def __init__(self, input_dim = (1, 28, 28), 
                 conv_param = {'filter_num' : 30, 'filter_size' : 5, 'pad' : 0, 'stride' : 1}, 
                 hidden_size = 100, output_size = 10, weight_init_std = 0.01):
        
        # hyper parameter 
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))
        
        # network parameter initialize
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)
        
        # layer construction
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(W = self.params['W1'], b=self.params['b1'], 
                                           stride=conv_param['stride'] , pad = conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h = 2, pool_w = 2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        
        self.lastLayer = SoftmaxWithLoss()
        
    # inferrence
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return(x)
    
    # loss computation
    def loss(self, x, t):
        y = self.predict(x)
        return( self.lastLayer.forward(y, t) )
    
    def accuracy(self, x, t, batch_size = 100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
            
        acc = 0.0
        
        for i in range(int(x.shape[0]/batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis =1)
            acc += np.sum(y == tt)
        
        return acc/x.shape[0]

        
    def gradient(self, x, t):
        # forward
        self.loss(x, t)
        
        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].db
        
        return grads
        

In [63]:
from keras.models import Sequential
from keras.layers.core import Flatten
from keras.layers import Dense, Activation, Reshape
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

# Conv-ReLU-Pooling-Dense-ReLU-Dense-Softmax

def KerasSimpleCNN():
    model = Sequential()
    model.add(Convolution2D(30, 5, 5, border_mode='same', input_shape=(1, 28, 28))) 
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Activation('relu'))
    model.add(Dense(10))
    model.add(Activation('softmax'))
    
    return(model)

Kerasmodel = KerasSimpleCNN()
Kerasmodel.compile(loss = 'categorical_crossentropy', optimizer = 'sgd', metrics=['accuracy'])
Kerasmodel.summary()


xs_train =  x_train[:5000]
ts_train = t_train[:5000]
Kerasmodel.fit(xs_train, ts_train, nb_epoch=10, batch_size=100)


    

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_11 (Convolution2D) (None, 30, 28, 28)    780         convolution2d_input_11[0][0]     
____________________________________________________________________________________________________
activation_27 (Activation)       (None, 30, 28, 28)    0           convolution2d_11[0][0]           
____________________________________________________________________________________________________
maxpooling2d_11 (MaxPooling2D)   (None, 30, 14, 14)    0           activation_27[0][0]              
____________________________________________________________________________________________________
flatten_9 (Flatten)              (None, 5880)          0           maxpooling2d_11[0][0]            
___________________________________________________________________________________________

In [64]:
Kerasmodel.evaluate(x_test, t_test, batch_size = 100)

10000/10000 [==============================] - 16s     

[0.35713314026594162, 0.89820000410079959]

In [60]:
# print(x_train.shape, t_train.shape, x_test.shape, t_test.shape)
from keras.datasets import mnist
from keras.utils import np_utils

(x_train, t_train), (x_test, t_test) = mnist.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train = x_train.reshape(x_train.shape[0], 1, x_train.shape[1], x_train.shape[2])
x_test = x_test.reshape(x_test.shape[0], 1, x_test.shape[1], x_test.shape[2])

nb_classes = 10
t_train = np_utils.to_categorical(y_train, nb_classes)
t_test = np_utils.to_categorical(y_test, nb_classes)

In [62]:
print(x_train.shape, t_train.shape, x_test.shape, t_test.shape)
x_train[:5000].shape

(60000, 1, 28, 28) (60000, 10) (10000, 1, 28, 28) (10000, 10)


(5000, 1, 28, 28)

In [ ]:
class TFSimpleCNN

In [16]:
from common.trainer import Trainer
from dataset.mnist import load_mnist

# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

# 処理に時間のかかる場合はデータを削減 
x_train, t_train = x_train[:5000], t_train[:5000]
x_test, t_test = x_test[:1000], t_test[:1000]

max_epochs = 20

network = ScratchSimpleCNN(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
                        
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=max_epochs, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# パラメータの保存
# network.save_params("params.pkl")
# print("Saved Network Parameters!")

# グラフの描画
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, trainer.train_acc_list, marker='o', label='train', markevery=2)
plt.plot(x, trainer.test_acc_list, marker='s', label='test', markevery=2)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()






train loss:2.29946550027
=== epoch:1, train acc:0.244, test acc:0.277 ===
train loss:2.29702502048
train loss:2.29406397712
train loss:2.28461287892
train loss:2.28008010528
train loss:2.26638945166
train loss:2.25837741492
train loss:2.23602969888
train loss:2.21086627905
train loss:2.20204173604
train loss:2.16168761305
train loss:2.13062420103
train loss:2.04999742307
train loss:2.01854182985
train loss:1.96381887256
train loss:1.99330975542
train loss:1.84697280325
train loss:1.74462169588
train loss:1.73287722
train loss:1.63741713724
train loss:1.51275040399
train loss:1.52062509288
train loss:1.32288468812
train loss:1.35152394121
train loss:1.25677241191
train loss:1.11409019567
train loss:1.05327770541
train loss:1.02051625908
train loss:0.866284108168
train loss:0.837533916862
train loss:0.906804860773
train loss:0.814655397225
train loss:0.82539976609
train loss:0.674342376557
train loss:0.559356867359
train loss:0.739801459804
train loss:0.715102575012
train loss:0.58992301

NameError: name 'plt' is not defined